In [1]:
pip install yfinance


In [ ]:
import yfinance as yf

# Function to calculate likelihood
def calculate_likelihood(data, x, bought_price):
    # Initialize variables to count successful cases and total cases
    success_count = 0
    total_count = 0

    # Iterate through each row in the dataframe starting from x+1
    for i in range(x + 1, len(data)):
        # Check if current price is 5% more than bought price
        if data['Close'][i] >= 1.05 * bought_price:
            # Calculate percentage increase from bought price to highest price in subsequent (1.95*x) days
            high_price = data['High'][i:i + int(1.95 * x)].max()
            if bought_price != 0:
                percentage_increase = (high_price - bought_price) / bought_price * 100

                # Check if percentage increase exceeds 10%
                if percentage_increase >= 10:
                    success_count += 1
                total_count += 1

    # Calculate likelihood
    likelihood = success_count / total_count if total_count != 0 else 0

    return likelihood

# Fetch historical data for a specified stock symbol from Yahoo Finance
def fetch_data(stock_symbol):
    try:
        stock_data = yf.download(stock_symbol, start="2022-01-01", end="2022-12-31")
        return stock_data
    except Exception as e:
        print("Error fetching data:", e)
        return None

# Main function
def main():
    # Specify the stock symbol (e.g., "SBIN.BO" for State Bank of India on NSE)
    stock_symbol = "SBIN.BO"

    # Fetch historical data
    data = fetch_data(stock_symbol)

    if data is not None:
        # Set the number of days (x)
        x = 25

        # Find the first occurrence when the current price is 5% more than the low of past 25 days
        bought_price = 0
        for i in range(x, len(data)):
            if data['Low'][i - x:i].min() * 1.05 <= data['Close'][i]:
                bought_price = data['Close'][i]
                break

        if bought_price != 0:
            # Calculate likelihood
            likelihood = calculate_likelihood(data, x, bought_price)
            print(f"Likelihood of achieving a 10% increase before {(1.95 * x)} days: {likelihood:.2%}")
        else:
            print("No buying opportunity found in the historical data.")
    else:
        print("Unable to fetch data. Please check the stock symbol.")

if __name__ == "__main__":
    main()


In [ ]:
pip install yfinance pandas


## **Likelihood Value**

In [ ]:
import yfinance as yf
import pandas as pd

def calculate_likelihood(ticker):
    # Fetch historical market data from Yahoo Finance for the last 3 years
    df = yf.download(ticker, period="3y", interval="1d")

    # Add a column for the 25-day low
    df['25d_low'] = df['Low'].rolling(window=25).min()

    # Initialize variables
    count = 0
    successful_count = 0
    bought_price = None
    x = 0

    # Traverse through the share prices
    for i in range(25, len(df)):
        current_price = df['Close'][i]
        current_low_25d = df['25d_low'][i]

        if bought_price is None and current_price <= 1.05 * current_low_25d:
            # Condition to buy the stock
            bought_price = current_price
            x = 0

        if bought_price is not None:
            x += 1

            if current_price >= 1.05 * bought_price:
                # Condition to mark the x days when 5% increase is reached
                count += 1
                target_days = int(1.95 * x)
                target_price = 1.10 * bought_price
                success = False

                # Check for the 10% increase within the target days
                for j in range(i + 1, min(i + 1 + target_days, len(df))):
                    if df['Close'][j] >= target_price:
                        successful_count += 1
                        success = True
                        break

                # Reset for new bought price if condition fails
                bought_price = None if success else current_price
                x = 0

    # Calculate likelihood
    likelihood = (successful_count / count) if count > 0 else 0
    return likelihood

# Example usage for an Indian stock (e.g., Reliance Industries)
ticker = 'RELIANCE.NS'
likelihood = calculate_likelihood(ticker)
print(f"The likelihood of the share price reaching 10% within 1.95x days after a 5% increase for {ticker} is: {likelihood:.2f}")


# **Likelihood with Count**

In [ ]:
import yfinance as yf
import pandas as pd

def calculate_likelihood(ticker):
    # Fetch historical market data from Yahoo Finance for the last 3 years
    df = yf.download(ticker, period="3y", interval="1d")

    # Add a column for the 25-day low
    df['25d_low'] = df['Low'].rolling(window=25).min()

    # Initialize variables
    count = 0
    successful_count = 0
    bought_price = None
    x = 0

    # Traverse through the share prices
    for i in range(25, len(df)):
        current_price = df['Close'][i]
        current_low_25d = df['25d_low'][i]

        if bought_price is None and current_price <= 1.05 * current_low_25d:
            # Condition to buy the stock
            bought_price = current_price
            x = 0

        if bought_price is not None:
            x += 1

            if current_price >= 1.05 * bought_price:
                # Condition to mark the x days when 5% increase is reached
                count += 1
                target_days = int(1.95 * x)
                target_price = 1.10 * bought_price
                success = False

                # Check for the 10% increase within the target days
                for j in range(i + 1, min(i + 1 + target_days, len(df))):
                    if df['Close'][j] >= target_price:
                        successful_count += 1
                        success = True
                        break

                # Reset for new bought price if condition fails
                bought_price = None if success else current_price
                x = 0

    # Calculate likelihood
    likelihood = (successful_count / count) if count > 0 else 0
    return likelihood, count, successful_count

# Example usage for an Indian stock (e.g., Reliance Industries)
ticker = 'RELIANCE.NS'
likelihood, count, successful_count = calculate_likelihood(ticker)
print(f"\nThe likelihood of the share price reaching 10% within 1.95x days after a 5% increase for {ticker} is: {likelihood:.2f}")
print(f"Number of times bought at 5% increase: {count}")
print(f"Number of times the price hit 10% before 1.95x: {successful_count}")


# **Likelihood with Count and Duration**

In [2]:
import yfinance as yf
import pandas as pd

def calculate_likelihood(ticker):
    # Fetch historical market data from Yahoo Finance for the last 3 years
    df = yf.download(ticker, period="5y", interval="1d")

    # Add a column for the 25-day low
    df['25d_low'] = df['Low'].rolling(window=25).min()

    # Initialize variables
    count = 0
    successful_count = 0
    bought_price = None
    x = 0
    successful_durations = []  # List to store successful durations (x values)
    unsuccessful_durations = []  # List to store unsuccessful durations (x values)

    # Traverse through the share prices
    for i in range(25, len(df)):
        current_price = df['Close'][i]
        current_low_25d = df['25d_low'][i]

        if bought_price is None and current_price <= 1.05 * current_low_25d:
            # Condition to buy the stock
            bought_price = current_price
            x = 0

        if bought_price is not None:
            x += 1

            if current_price >= 1.05 * bought_price:
                # Condition to mark the x days when 5% increase is reached
                count += 1
                target_days = int(1.95 * x)
                target_price = 1.10 * bought_price
                success = False

                # Check for the 10% increase within the target days
                for j in range(i + 1, min(i + 1 + target_days, len(df))):
                    if df['Close'][j] >= target_price:
                        successful_count += 1
                        successful_durations.append(x)  # Store the successful duration (x value)
                        success = True
                        break

                if not success:
                    unsuccessful_durations.append(x)  # Store the unsuccessful duration (x value)

                # Reset for new bought price if condition fails
                bought_price = None if success else current_price
                x = 0

    # Calculate likelihood
    likelihood = (successful_count / count) if count > 0 else 0
    return likelihood, count, successful_count, successful_durations, unsuccessful_durations

# Example usage for an Indian stock (e.g., Reliance Industries)
ticker = 'KAYNES.NS'
likelihood, count, successful_count, successful_durations, unsuccessful_durations = calculate_likelihood(ticker)
print(f"\nThe likelihood of the share price reaching 10% within 1.95x days after a 5% increase for {ticker} is: {likelihood:.2f}")
print(f"Number of times bought at 5% increase: {count}")
print(f"Number of times the price hit 10% before 1.95x: {successful_count}")
print(f"Durations (x) for successful attempts: {successful_durations}")
print(f"Durations (x) for unsuccessful attempts: {unsuccessful_durations}")


[*********************100%%**********************]  1 of 1 completed


The likelihood of the share price reaching 10% within 1.95x days after a 5% increase for KAYNES.NS is: 0.67
Number of times bought at 5% increase: 6
Number of times the price hit 10% before 1.95x: 4
Durations (x) for successful attempts: [9, 8, 7, 6]
Durations (x) for unsuccessful attempts: [7, 34]
